In [2]:
import numpy as np, pandas as pd
import re
import twobitreader
import math

# The cells below detail how to annotate a call stats file to be used as a training set

## Before running, make sure that the call stats are annotated for overlaps with segmental duplications, germline copy number gains and have UCSC mappability scores. Filtering out overlap with Hardy Weinberg failures also recommended

In [2]:
input_filename="cs.txt"
output_filename="cs_af_next.txt"

train=pd.read_csv(input_filename,sep='\t')

train=train.sort_values(by=['contig',"position"])

train=train.reset_index(drop=True)

train['af_next']=train['tumor_f']
train['af_prev']=train['tumor_f']

#get the allele fractions of the next SNP
train['af_next']=train['af_next'].shift(periods=-1)

#get allele fractions of the previous SNP
train['af_prev']=train['af_prev'].shift()

train.to_csv(output_filename,sep='\t',index=False)

FileNotFoundError: File b'all_clone/blacklist/exac_cluster_allClone_hwe_segdup_CNV.maf' does not exist

In [3]:
#rename alphabetical chromosomes to numbers
def chrom2int(x):
    if x == 'X':
        return 23
    elif x == 'Y':
        return 24
    else:
        return int(x)

In [78]:
## Annotating training file for the distribution of allele fractions at a given site and the distance to the
## nearest SNP
input_filename="cs_af_next.txt"
train=pd.read_csv(input_filename,sep='\t')
train['contig']=train['contig'].apply(str)

pattern=r'^[0-9]+$'
pattern_X=r'^X$'
pattern_Y=r'^Y$'

train=train.loc[(train["contig"].str.contains(pattern))|(train["contig"].str.contains(pattern_X))|
                (train["contig"].str.contains(pattern_Y)),:]
train.reset_index(drop=True,inplace=True)
print(len(train.index))


chr_lens = [0, 249250621, 243199373, 198022430, 191154276, 180915260, 171115067, 159138663,
              146364022, 141213431, 135534747, 135006516, 133851895, 115169878, 107349540,
              102531392, 90354753, 81195210, 78077248, 59128983, 63025520, 48129895, 51304566,
           155270560]

offsets = np.cumsum(chr_lens)

chrom=[]

for idx,row in train.iterrows():
    chrom.append(chrom2int(row.contig))

train["Chromosome"]=chrom
train["genomic_position"]=pd.to_numeric(train["position"])+offsets[
    pd.to_numeric(train["Chromosome"])-1
]

train['af_category']=train['tumor_f']//0.1

cat_counts=train.groupby(["genomic_position","af_category"]).size().reset_index(name='counts')
cat_counts=cat_counts.pivot(index='genomic_position',columns='af_category',values='counts')
cat_counts=cat_counts.fillna(0)

cat_counts['row_sum']=cat_counts.iloc[:,0:len(cat_counts.columns)].sum(axis=1)
cat_counts=cat_counts.iloc[:,0:len(cat_counts.columns)-1].div(cat_counts.loc[:,'row_sum'],axis=0)
cat_counts['genomic_position']=cat_counts.index

train=train.merge(cat_counts,left_on="genomic_position",right_on="genomic_position")


train.rename(columns={0.0: "less_than_0.1", 1.0: "between_0.1_0.2",
                                     2.0:"between_0.2_0.3",3.0:"between_0.3_0.4",
                                     4.0:"between_0.4_0.5",5.0:"between_0.5_0.6"},inplace=True)

mean_val=train.groupby(["genomic_position"])['tumor_f'].mean().reset_index(name='tumor_f')
print(len(mean_val.index))

train=mean_val.merge(train,left_on="genomic_position",right_on="genomic_position")
train=train.reset_index(drop=True)

train=train.drop_duplicates(subset='genomic_position', keep='first')

train=train.sort_values(by=["genomic_position"])

train['distance_prev']=train['genomic_position'].diff()
train['distance_next']=abs(train['genomic_position'].diff(periods=-1))
print(train.contig.unique())

print([np.max(train['distance_prev']),np.max(train['distance_next'])])
print(len(train.index))

train.to_csv("small_case_shuffled_HWE_segdup_commonCN_map_proxy_dist.txt",sep='\t',index=False)


66893


/Users/mleventh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: 'genomic_position' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


34225
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' 'X' 'Y']
[21504264.0, 21504264.0]
34225


# The following cells can be used to either annotate a training set or annotate a somatic mutation calling set for SVM prediciton. 

## Like the call stats file, the somatic mutation calling calling file should be annotated for overlaps with segmental duplications, germline CNVs and UCSC mappability scores

In [5]:
## annotate for proximity to bait boundary
train=pd.read_csv("test.txt",sep='\t')
agilent=pd.read_csv("whole_exome_agilent_1.1_refseq_plus_3_boosters.Homo_sapiens_assembly19.targets.bed",sep='\t')

chrom=[]

for idx,row in train.iterrows():
    #run if using a call stats file
    #chrom.append(chrom2int(row.contig))
    
    #run if using an annotated maf
    chrom.append(chrom2int(row.Chromosome))

train["Chromosome_num"]=chrom

chrom=[]
for idx,row in agilent.iterrows():
    chrom.append(chrom2int(row.Chromosome))

agilent["Chromosome"]=chrom

agilent=agilent.dropna(subset=["Chromosome","Start_position","End_position"],axis=0)
bait_boundary=[]
for idx,row in train.iterrows():
    
    agilent_chrom=agilent.loc[agilent['Chromosome']==row.Chromosome_num,:]

    #for call stats
    #diff_start=abs(agilent_chrom.Start_position-row.position)
    #diff_end=abs(agilent_chrom.End_position-row.position)
    
    #for annotated maf
    diff_start=abs(agilent_chrom.Start_position-row.Start_position)
    diff_end=abs(agilent_chrom.End_position-row.Start_position)
    
    boundary=min(min(diff_start),min(diff_end))

    bait_boundary.append(boundary)
    
train['distance_to_bait']=bait_boundary

train.to_csv("test_bait.txt",sep='\t',index=False)
    

/Users/mleventh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
## annotate GC content of 75mer context, can be slow
def chrom2int(x):
    if x == 'X':
        return 23
    elif x == 'Y':
        return 24
    else:
        return int(x)

def get_reference(genome,chrom,start,end):

    contig=genome[str(chrom)]
    return contig[start:end+1]

def find_reads(chrm,pos,rep_len):

    # initialize 2bitreader genome
    genome = twobitreader.TwoBitFile("hg19.2bit")
    chr=[genome[c] for c in map(str,range(1,23))]
    chr.append(genome['X'])
    chr.append(genome['Y'])

    read=get_reference(genome,chrm,pos-(rep_len//2)-1,pos+(rep_len//2)-1)
    
    return read

def gc_content(read):
    return (read.count('G')+read.count('C'))/len(read)

train=pd.read_csv("test_bait.txt",sep='\t')
train['gc_content']=0

for idx,row in train.iterrows():
    #gc_read=find_reads(row['contig'],row['position'],75)
    gc_read=find_reads(row['Chromosome'],row['Start_position'],75)
    train.loc[idx,'gc_content']=gc_content(gc_read)

train.to_csv("test_gc.txt",
             sep='\t',index=False)

ANNOTATE SOMATIC MUTATIONS WITH NEAREST GERMLINE AF, POSITION, AND RECURRENCE

In [12]:
germline_file="call_stat_training/small_case_shuffled_medians_segdup_CNV_proxy_map_dist_bait_gc.txt"
somatic_file="test_gc.txt"

germline_muts=pd.read_csv(germline_file,sep='\t')
somatic_muts=pd.read_csv(somatic_file,sep='\t')

chr_lens = [0, 249250621, 243199373, 198022430, 191154276, 180915260, 171115067, 159138663,
              146364022, 141213431, 135534747, 135006516, 133851895, 115169878, 107349540,
              102531392, 90354753, 81195210, 78077248, 59128983, 63025520, 48129895, 51304566,
           155270560]

offsets = np.cumsum(chr_lens)

germ_chrom=[]
som_chrom=[]

#make sure all contigs are numerical
for idx,row in germline_muts.iterrows():
    germ_chrom.append(chrom2int(row.contig))
    
for idx,row in somatic_muts.iterrows():
    som_chrom.append(chrom2int(row.Chromosome))

#calculate genomic position
germline_muts["Chromosome"]=germ_chrom
somatic_muts['numbered_Chromosome']=som_chrom
germline_muts["genomic_position"]=pd.to_numeric(germline_muts["position"])+offsets[
    pd.to_numeric(germline_muts["Chromosome"])-1
]
                                         
somatic_muts["genomic_position"]=pd.to_numeric(somatic_muts["Start_position"])+offsets[
    pd.to_numeric(somatic_muts["numbered_Chromosome"])-1
]

#sort and combine
germline_muts=germline_muts.sort_values(by='genomic_position')
germline_muts=germline_muts.reset_index(drop=True)

somatic_muts=somatic_muts.sort_values(by='genomic_position')
somatic_muts=somatic_muts.reset_index(drop=True)

somatic_unique=somatic_muts.drop_duplicates(subset=["genomic_position"])

merge_muts=pd.merge_asof(somatic_unique,germline_muts,on='genomic_position',allow_exact_matches=True)
merge_muts['SNP_genomic_position']=pd.to_numeric(merge_muts["position"])+offsets[
    pd.to_numeric(merge_muts["Chromosome_y"])-1
]

#get distance to the nearest SNPs
merge_muts['distance_prev']=merge_muts['genomic_position']-merge_muts['SNP_genomic_position']

merge_muts['distance_next']=abs(merge_muts['genomic_position']-merge_muts['SNP_genomic_position'].shift(-1))

#If there is no SNP at a distance greater than the last somatic mutation, calculate distance to the end of the genome
merge_muts.loc[len(merge_muts.index)-1,'distance_next']=3088286401-merge_muts.loc[len(merge_muts.index)-2,'genomic_position']

#failsafe to check if any of the merges were performed incorrectly
negative=np.where(merge_muts['distance_prev']<0)[0]
print(negative)

somatic_merge=somatic_muts.merge(merge_muts,on="genomic_position")

somatic_merge.to_csv("test_prepared.txt",sep='\t',index=False)

Index(['tumor_f', 'ref_allele', 'context', 'contig', 'position', 'hwe_fail',
       'has_segdup', 'gain_overlap', 'loss_overlap', 'score', 'af_next',
       'af_prev', 'less_than_0.1', 'between_0.1_0.2', 'between_0.2_0.3',
       'between_0.3_0.4', 'between_0.4_0.5', 'between_0.5_0.6',
       'distance_prev', 'distance_next', 'Chromosome', 'distance_to_bait',
       'gc_content'],
      dtype='object')
Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'Center', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele1',
       'Tumor_Seq_Allele2', 'Protein_Change', 'Cluster_ID', 'CCF_hat_01',
       'CCF90_low_01', 'CCF90_high_01', 'Tumor_Sample_Barcode',
       'germline_cluster', 'hwe_fail', 'gain_overlap', 'has_segdup',
       't_alt_count', 't_ref_count', 'SwissProt_acc_Id', 'SwissProt_entry_Id',
       'ref_context', 'pon_loglike', 'i_t_ALT_F1R2', 'i_t_ALT_F2R1',
       'i_t_REF_F1R2